In [2]:
from utils.utils import load_config
from datasets.SyntheticData import SyntheticData
import torch

## two-step-SINDy

In [3]:
import pandas as pd
import numpy as np 

def NumericalDeriv(TimeSeries,dim,Nnodes,deltT):
    x_center = TimeSeries[2:-2,:]
    x_PlusTwo = TimeSeries[4:,:]
    x_PlusOne = TimeSeries[3:-1,:]
    x_MinusTwo = TimeSeries[:-4,:]
    x_MinusOne = TimeSeries[1:-3,:]
    dxdt = (x_MinusTwo - 8 * x_MinusOne + 8 * x_PlusOne - x_PlusTwo) / (12 * deltT)
    T_len = len(dxdt[:,0])
    NumDiv = np.zeros(shape=(T_len*Nnodes,dim))
    for j in range(0,dim):
        for i in range(0,Nnodes):
            NumDiv[i*T_len:(i+1)*T_len,j]  = dxdt[:,dim*i+j]
    if dim == 1:
        column_values = ['dx1']
    if dim == 2:
        column_values = ['dx1','dx2']
    if dim == 3:
        column_values = ['dx1','dx2','dx3']
    if dim == 4:
        column_values = ['dx1','dx2','dx3','dx4']
    NumDiv= pd.DataFrame(data = NumDiv, columns = column_values)
    return NumDiv

def preprocess(FuncMatrix,NumDiv,dim):
    preprocess = pd.concat([FuncMatrix,NumDiv],axis = 1)
    preprocess = preprocess.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    FuncMatrix = preprocess.iloc[:,0:FuncMatrix.shape[1]]
    NumDiv = preprocess.iloc[:,-dim:]
    return FuncMatrix,NumDiv

In [ ]:
# import pandas as pd
# import numpy as np
# import sys

# def ElementaryFunctions_Matrix(TimeSeries, dim, Nnodes, A, selfPolyOrder, coupledPolyOrder = 1, PolynomialIndex = True, TrigonometricIndex = True, \
#     ExponentialIndex = True, FractionalIndex = True, ActivationIndex = True, RescalingIndex = True, CoupledPolynomialIndex = True, \
#         CoupledTrigonometricIndex = True, CoupledExponentialIndex = True, CoupledFractionalIndex = True, \
#             CoupledActivationIndex = True, CoupledRescalingIndex = True):
    
#     ElementaryMatrix = pd.DataFrame()
#     if PolynomialIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix,Polynomial_functions(TimeSeries, dim, Nnodes, selfPolyOrder)],axis=1)
#     if TrigonometricIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Trigonometric(TimeSeries, dim, Nnodes, Sin = True, Cos = True, Tan = True)],axis=1)
#     if ExponentialIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Exponential(TimeSeries, dim, Nnodes, expomential = True)],axis=1)
#     if FractionalIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Fractional(TimeSeries, dim, Nnodes, fractional = True)],axis=1)
#     if ActivationIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Activation(TimeSeries, dim, Nnodes, Sigmoid = True, Tanh = True, Regulation = True)],axis=1)
#     if RescalingIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, rescaling(TimeSeries, dim, Nnodes, A, Rescal = True)],axis=1)
#     if CoupledPolynomialIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, coupled_Polynomial_functions(TimeSeries, dim, Nnodes, A, coupledPolyOrder)],axis=1)
#     if CoupledTrigonometricIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Coupled_Trigonometric_functions(TimeSeries, dim, Nnodes, A, Sine = True, Cos = False, Tan = False)],axis=1)
#     if CoupledExponentialIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Coupled_Exponential_functions(TimeSeries, dim, Nnodes, A, Exponential = True)],axis=1)
#     if CoupledFractionalIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Coupled_Fractional_functions(TimeSeries, dim, Nnodes, A, Fractional = True)],axis=1)
#     if CoupledActivationIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Coupled_Activation_functions(TimeSeries, dim, Nnodes, A, Sigmoid = True, Tanh = True, Regulation = True)],axis=1)
#     if CoupledRescalingIndex == True:
#         ElementaryMatrix = pd.concat([ElementaryMatrix, Coupled_Rescaling_functions(TimeSeries, dim, Nnodes, A, Rescaling = True)],axis=1)
        
#     return ElementaryMatrix


## Load data

In [4]:
config = load_config(config_path='./configs/config_pred_deriv/config_ic3/config_kuramoto.yml')

dataset = SyntheticData(
    root=config['data_folder'],
    dynamics=config['name'],
    t_span=config['t_span'],
    t_max=config['t_eval_steps'],
    num_samples=config['num_samples'],
    seed=config['seed'],
    n_ics=config['n_iter'],
    input_range=config['input_range'],
    device=config['device'],
    horizon = config['horizon'],
    history = config['history'],
    stride=config.get('stride', 5),
    predict_deriv=config.get("predict_deriv", False),
    **config['integration_kwargs']
)

## Data and NumDeriv

In [ ]:
raw_data = dataset.raw_data_sampled.cpu().detach().numpy() # shape: (ics, time_steps, n_nodes, 1)
time = dataset.t_sampled
dt = time[0, 1] - time[0, 0]
dt = dt.item()

data = []
num_deriv = []

for ic in range(raw_data.shape[0]):
    data_ic = raw_data[ic].squeeze(-1)  # shape: (time_steps, n_nodes)
    num_deriv_ic = NumericalDeriv(
        TimeSeries=data_ic,
        dim=1,
        Nnodes=data_ic.shape[1],
        deltT=dt
    )   # pd DatafRame
    
    # TODO: sample data and num_deriv with stride s
    
    data.append(data_ic[2:-2,:])
    num_deriv.append(num_deriv_ic)
    

data = np.concatenate(data, axis=0)
num_deriv = pd.concat(num_deriv, ignore_index=True)

## Adj Matrix

In [9]:
edge_index = dataset[0].edge_index

In [14]:
from torch_geometric.utils import to_dense_adj

A = to_dense_adj(edge_index)[0].cpu().detach().numpy()